In [1]:
import os
from tbparse import SummaryReader

# log_dir = os.path.expanduser("~/lis-cluster/emergent_communication/emergent_communication/lightning_logs")
log_dir = os.path.expanduser("~/PhD/emergent_communication/emergent_communication/lightning_logs_cluster")

reader = SummaryReader(log_dir, extra_columns={'dir_name'}, pivot=True)
df = reader.scalars
hp = reader.hparams

# Save backups:
df_original = df.copy()
hp_original = hp.copy()

In [2]:
df = df_original.copy()
hp = hp_original.copy()

In [3]:
def fix_duplicate_value(val):
    if isinstance(val, list):
        for el in val:
            assert (el == val[0]) or (el == "None") or (val[0] == "None")
        return val[0]
    else:
        return val

hp = hp.applymap(fix_duplicate_value)
# df = df.applymap(fix_duplicate_value)  # Takes too long

In [4]:
# Remove duplicate logs
df.dropna(subset=["val_acc_no_noise", "topsim"], inplace=True)
unfinished_runs = df[df["val_acc_no_noise"].apply(isinstance, args=(list, ))]
print(f"Duplicate runs: {unfinished_runs.dir_name.unique()}")
df = df[~df["val_acc_no_noise"].apply(isinstance, args=(list,))].copy()

Duplicate runs: []


In [5]:
# Keep only best step values for each run
df["val_acc_no_noise"] = df["val_acc_no_noise"].astype(float)
indices_best_steps = df.groupby(["dir_name"])["val_acc_no_noise"].idxmax()
df = df.loc[indices_best_steps]
df

,step,best_val_acc_no_noise,bosdis,bosdis_at_best_val_acc,entropy_loss,entropy_loss/dataloader_idx_0,entropy_loss/dataloader_idx_1,entropy_loss/dataloader_idx_2,epoch,hp_metric,...,test_acc,test_acc_no_noise,topsim,topsim_at_best_val_acc,train_acc,train_acc_no_noise,train_acc_no_noise_at_best_val_acc,val_acc,val_acc_no_noise,dir_name
50,2499,1.0,0.110944,0.110944,0.094175,0.097214,0.086236,0.096345,"[2499.0, 2499.0]",NaN,...,0.884615,0.884615,0.567532,0.567532,0.985507,1.0,1.0,1.0,1.0,version_1053667
295,7199,1.0,0.061478,0.061478,0.362947,0.359391,0.367527,0.365527,"[7199.0, 7199.0]",NaN,...,1.0,1.0,0.554759,0.554759,0.990338,1.0,1.0,0.913043,1.0,version_1053668
564,8399,1.0,0.113361,0.113361,0.138065,0.141128,0.129169,0.138643,"[8399.0, 8399.0]",NaN,...,0.923077,0.923077,0.521529,0.521529,0.995169,1.0,1.0,1.0,1.0,version_1053669
751,4299,1.0,0.093255,0.093255,0.348223,0.342142,0.356651,0.349823,"[4299.0, 4299.0]",NaN,...,0.961538,1.0,0.478973,0.478973,0.975845,1.0,1.0,0.913043,1.0,version_1053670
926,3699,1.0,0.084773,0.084773,0.10164,0.112596,0.109427,0.108363,"[3699.0, 3699.0]",NaN,...,0.923077,0.923077,0.544981,0.544981,0.975845,1.0,1.0,1.0,1.0,version_1053671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410942,2700,1.0,0.085624,0.085624,NaN,0.331938,0.332617,0.336752,2700.0,NaN,...,0.952381,0.984127,0.532374,0.532374,NaN,1.0,1.0,0.947368,1.0,version_860
411014,3500,1.0,0.089479,0.089479,NaN,0.342205,0.334754,0.335841,3500.0,NaN,...,0.984127,1.0,0.401707,0.401707,NaN,1.0,1.0,0.964912,1.0,version_861
411068,2600,1.0,0.084645,0.084645,NaN,0.319353,0.3252,0.327036,2600.0,NaN,...,0.968254,0.968254,0.407298,0.407298,NaN,0.994059,0.994059,0.964912,1.0,version_862
411126,2800,1.0,0.099954,0.099954,NaN,0.337219,0.335558,0.332906,2800.0,NaN,...,0.968254,0.984127,0.459714,0.459714,NaN,0.99802,0.99802,0.929825,1.0,version_863


In [6]:
df.set_index("dir_name", inplace=True, drop=False, verify_integrity=True)
hp.set_index("dir_name", inplace=True, verify_integrity=True)

df = df.join(hp, how="left")

In [7]:
df.to_pickle("df.pickle")